In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/cheese_classification_challenge-master

# Requirements

In [ ]:
import PIL
from PIL import Image
import os
from IPython.display import display
import random
import numpy as np
from pathlib import Path

In [ ]:
!pip install -r requirements.txt
!pip install google.cloud
!pip install google.cloud.vision
!pip install ultralytics
!pip install xformers

# DATA AUGMENTATION

les fonctions de data augmentation suivent toutes la même logique de fonctionnement :
- elles prennent en entrée une liste de path vers des images
- ces images sont au format 768 * 768
- elles retournent la liste des images process correspondantes

In [ ]:
def crop_image(image):
    # Calculate the maximum top-left corner for a 400x400 crop
    image = image.resize((768,768))
    max_x = 768 - 450
    max_y = 768 - 450


          # Randomly select the top-left corner
    left = random.randint(0, max_x)
    top = random.randint(0, max_y)
    right = left + 450
    bottom = top + 450

          # Crop the image
    cropped_image = image.crop((left, top, right, bottom))

    return cropped_image

In [ ]:
def crop_images_1024(image, multiplicateur):
    cropped_images = []


        # Ensure the image is in 768x768 format
    if image.size != (1024, 1024):
        raise ValueError(f"Image at {path} is not in 768x768 format")

        # Calculate the maximum top-left corner for a 400x400 crop
    max_x = 1024 - 300
    max_y = 1024 - 300

    for i in range(multiplicateur):
      # Randomly select the top-left corner
      left = random.randint(0, max_x)
      top = random.randint(0, max_y)
      right = left + 300
      bottom = top + 300

          # Crop the image
      cropped_image = image.crop((left, top, right, bottom))

          # Add the cropped image to the list
      cropped_images.append(cropped_image)

    return cropped_images

In [ ]:
def image_with_noise_768(image, std=25, size=300):
    images_with_noise = []
    if image.size != (768, 768):
        raise ValueError(f"Image at {path} is not in 768x768 format")

    image = np.array(image)
    noise = np.random.normal(0, std, (size, size, 3)).astype(np.uint8)

    noise = Image.fromarray(noise)

    resized_noise = noise.resize((768, 768))

    resized_noise = np.array(resized_noise)

        # Add the noise to the image
    noisy_image = image + resized_noise

        # Clip the values to be in the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

        # Convert back to PIL image
    noisy_image = Image.fromarray(noisy_image)

    images_with_noise.append(noisy_image)

    return images_with_noise

In [ ]:
def image_with_noise_1024(image, std=25, size=400):
    images_with_noise = []

        # Ensure the image is in 768x768 format
    if image.size != (1024, 1024):
         raise ValueError(f"Image at {path} is not in 768x768 format")

    image = np.array(image)
    noise = np.random.normal(0, std, (size, size, 3)).astype(np.uint8)

    noise = Image.fromarray(noise)

    resized_noise = noise.resize((1024, 1024))

    resized_noise = np.array(resized_noise)

        # Add the noise to the image
    noisy_image = image + resized_noise

        # Clip the values to be in the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

        # Convert back to PIL image
    noisy_image = Image.fromarray(noisy_image)

    images_with_noise.append(noisy_image)

    return images_with_noise

In [ ]:
def grey_image(image):
    return image.convert('L')

In [ ]:
def downscaled_image(image, factor=0.3):
  new_width = int(image.width * factor)
  new_height = int(image.height * factor)
  new_size = (new_width, new_height)
  downscaled_image = image.resize(new_size)
  ret_image = downscaled_image.resize(image.size)
  return ret_image

# PIPELINE DE DATA AUGMENTATION

- pour chaque image, rajouter 3 sous crops de l'image, puis les resize au meme format

In [ ]:
base_path = '/content/drive/MyDrive/cheese_classification_challenge-master/dataset/train'
for item in os.listdir(base_path):
    print(os.path.join(base_path,item))

In [ ]:
def process_in_folder(folder_path):
  img_names = os.listdir(folder_path).copy()
  for img_name in img_names:
    # on ajoute une image crop, une image crop en niveau de gris, et l'image de base downsampler
    img_name = os.path.splitext(img_name)[0]
    img_path = os.path.join(folder_path, img_name) +'.png'
    print(img_path)
    img_init = Image.open(img_path)

    crop_img = crop_image(img_init)
    grey_img = grey_image(crop_image(img_init))
    downs_img = downscaled_image(crop_image(img_init), factor=0.2)

    path = img_path.replace('.png', '')
    path1 = path + 'crop' + '.png'

    path2 = path  + 'grey' + '.png'
    path3 = path  + 'downs' + '.png'

    crop_img.save(path1)
    grey_img.save(path2)
    downs_img.save(path3)

In [ ]:
for item in os.listdir(base_path):
    # Check if the item is a directory
    if os.path.isdir(os.path.join(base_path, item)):
        print("Folder:", item)
        label = item
        folder_dir = '/content/drive/MyDrive/cheese_classification_challenge-master/dataset/merge/' + label
        i = 0
        path_save = '/content/drive/MyDrive/cheese_classification_challenge-master/merge/' + label
        if not os.path.exists(path_save):
          os.mkdir(path_save)
        images = Path(folder_dir).glob('*.png')
        for image_path in images:
            image = Image.open(image_path)
            image_array=np.array(image)
            if(image_array.shape == (768, 768, 3)):
              img2 = crop_images_768(image, 4)
              img4 = grey_images(image)
              for im in img2:
                im.save(path_save + '/im_augmented_crop' + str(i) + '.png')
              for im in img4:
                im.save(path_save + '/im_augmented_black' + str(i) + '.png')
              i+= 1
            if(image_array.shape == (1024, 1024, 3)):
              img2 = crop_images_1024(image, 4)
              img4 = grey_images(image)
              for im in img2:
                im.save(path_save + '/im_augmented_crop' + str(i) + '.png')
              for im in img4:
                im.save(path_save + '/im_augmented_black' + str(i) + '.png')
              i+= 1
        print(i)

Folder: CABECOU
158
Folder: CAMEMBERT
163
Folder: CHABICHOU
115
Folder: CHEDDAR
168
Folder: CHÈVRE
162
Folder: COMTÉ
143
Folder: EMMENTAL
108
Folder: EPOISSES
146
Folder: FETA
150
Folder: FOURME DAMBERT
156
Folder: FROMAGE FRAIS
123
Folder: MAROILLES
151
Folder: MIMOLETTE
196
Folder: MONT D’OR
117
Folder: MOTHAIS
136
Folder: MOZZARELLA
176
Folder: MUNSTER
127
Folder: NEUFCHATEL
133
Folder: OSSAU- IRATY
186
Folder: PARMESAN
146
Folder: PECORINO
167
Folder: RACLETTE
121
Folder: POULIGNY SAINT- PIERRE
113
Folder: SAINT-NECTAIRE
174
Folder: SCARMOZA
112
Folder: STILTON
160
Folder: TÊTE DE MOINES
157
Folder: TOMME DE VACHE
141
Folder: VACHERIN
159
Folder: MORBIER
160
Folder: ROQUEFORT
198
Folder: REBLOCHON
152
Folder: SAINT- FÉLICIEN
176
Folder: BRIE DE MELUN
154
Folder: GRUYÈRE
122


# Create submission

In [ ]:
!python create_submition_with_ocr.py model=dinov2